In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File"  # file without extention
sheets = "85 Sentence"  # 2 gram target, 2 gram hybrid
file_ext = "85 Sentence"
first_column = 2  # start column location number
end_column = 3

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# word all usage in sent
word_use_big_than = 1

In [4]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [5]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [7]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [8]:
df_file = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name=f"{sheets}")
#df_file.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=True)
df_file.reset_index(drop=True, inplace=True)
df_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
1,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"
2,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
3,çok iyi değil mi tam böyle gerçek gibi gerçek ...,3,6,çok iyi değil mi tam böyle gerçek gibi gerçek ...,OmT_BtZh8qU,https://www.youtube.com/watch?v=OmT_BtZh8qU&t=3s,"[9, 28, 14, 5, 112, 56, 183, 27, 183, 27, 60]"
4,nasıl bir şekilde olduğunu ve neden o şekilde ...,14,17,şimdi apexin nasıl bir şekilde olduğunu ve ned...,owj1QZoYqFs,https://www.youtube.com/watch?v=owj1QZoYqFs&t=14s,"[39, 0, 181, 40, 3, 34, 6, 181, 40]"
...,...,...,...,...,...,...,...
80,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
81,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"
82,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,https://www.youtube.com/watch?v=mHkBxBLAZww&t=...,"[48, 81, 48, 81]"
83,bir gece çok kötü bir,822,824,hayat mücadelesi verdiğimde bir gece çok kötü ...,eS84Pe23HMA,https://www.youtube.com/watch?v=eS84Pe23HMA&t=...,NaN


In [9]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > word_use_big_than]
check_list = df_check["word"].to_list()
remove_list = []
df_file_var = df_file
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file_var = df_file_var[df_file_var["search_string"] != f"{var_sent}"]
        df_file_var_count = word_count_result(df_file_var, df_file_var.columns[0:1])
        df_check = df_file_var_count[df_file_var_count["word_count"] > word_use_big_than]
        check_list = df_check["word"].to_list()
        remove_list.append(var_sent)
    else:
        pass

df_file_var.reset_index(drop=True, inplace=True)
df_file_var

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
1,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"
2,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
3,bile yok bu da benim için aslında önemli bir ş...,131,134,bulabilirsiniz şu an görünce bile yok bu da be...,aqQ7IUho9pA,https://www.youtube.com/watch?v=aqQ7IUho9pA&t=...,"[59, 22, 1, 15, 31, 4, 174, 156, 0, 16, 68]"
4,için artık değil misin gerek yok bunun için si...,65,70,göndermek için artık değil misin gerek yok bun...,gX77n13Qj0o,https://www.youtube.com/watch?v=gX77n13Qj0o&t=65s,"[4, 61, 14, 49, 117, 22, 97, 4, 121, 0]"
...,...,...,...,...,...,...,...
57,diye söyle bak hala,104,106,o taşlarını gecesi zor için yoğurduktan sonra ...,P5bYtFrqw68,https://www.youtube.com/watch?v=P5bYtFrqw68&t=...,"[123, 161, 64, 167]"
58,lütfen ilk önce onu,21,23,videoyu izlemediyseniz lütfen ilk önce onu izl...,DWIzzc0OaIM,https://www.youtube.com/watch?v=DWIzzc0OaIM&t=21s,"[58, 193, 51, 23]"
59,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
60,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"


In [10]:
remove_list

['çok iyi değil mi tam böyle gerçek gibi gerçek gibi yani',
 'nasıl bir şekilde olduğunu ve neden o şekilde olduğunu',
 'burada böyle hayır öyle bir şey olabilir mi asla asla',
 'bence çok güzel bence çok güzel senin zaten',
 'biraz daha devam et devam et tamam oldu',
 'kız seni gel buraya gel buraya kız gel',
 'bakalım son olarak neler var neler',
 'hemen ve her gün her dakika oluyor',
 'aslında seninle birlikte orada',
 'bakalım nasıl olacak ve buna',
 'çünkü hiçbir zaman bana göre',
 'ile hemen hemen aynı hiçbir',
 'onları biliyor musun çünkü',
 'size söyle merhaba merhaba',
 'geldi ona kadar ona kadar',
 'olsun ki sadece biz bize',
 'bize lazım olacak çünkü',
 'biri büyük biri küçük',
 'tabii onlar tabii ona',
 'olan oldu artık geri',
 'tam başka en uzun',
 'şu adam şu adam',
 'tamam peki pekala pekala tamam']

In [11]:
df_result_word_count = word_count_result(df_file_var, df_file.columns[0:1])
df_result_word_count

,word,word_count
0,bir,13
1,çok,11
2,değil,7
3,için,7
4,bakalım,6
...,...,...
187,herkes,1
188,hemen,1
189,hayır,1
190,haydi,1


In [16]:
df_result_word_count["word_count"].sum()

395

In [17]:
df_file_var.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample_Selected.xlsx", index=False)

##### Not In Sent And Result Word Count

In [12]:
set_result_word_count = set(df_result_word_count["word"])
set_word_select = set(df_word_select["word"])

In [13]:
not_in_sent_word_list = list(set_word_select.difference(set_result_word_count))
not_in_sent_word_list

['bence', 'merhaba', 'adam', 'gerçek', 'asla', 'et', 'devam', 'pekala']

In [14]:
df_youtube_sent_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment2/Result/\
2-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...
...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN


In [15]:
not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_result, "search_string", 1, simple=False)
not_in_sent_selected_word_list_result = not_in_sent_selected_word_list_result.drop(["word"], axis=1)
not_in_sent_selected_word_list_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,o kadar güzel ki olmaz diye bir şey yok oluyor...,718,725,yağ gibi kayıyor o kadar güzel ki olmaz diye b...,p4E0s20uabE,https://www.youtube.com/watch?v=p4E0s20uabE&t=...
1,çok güzel burada sen burada merhaba sen bizim,1887,1891,uğraşıyor survivorda merhaba serman seni görme...,9GbYnL0tvcg,https://www.youtube.com/watch?v=9GbYnL0tvcg&t=...
2,bir söyle de bir bakalım o adam mı değil mi be...,5659,5663,sen hele bir söyle de bir bakalım o adam mı de...,XENntT235g4,https://www.youtube.com/watch?v=XENntT235g4&t=...
3,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...
4,burada böyle hayır öyle bir şey olabilir mi as...,489,493,mesela yüzde doksan dokuz deyip burada böyle h...,GZG9nDbIgXM,https://www.youtube.com/watch?v=GZG9nDbIgXM&t=...
5,baba ne olur yardım et bana ne olur yardım et ...,317,322,baba ne olur yardım et bana ne olur yardım et ...,a4UF4C1hiB0,https://www.youtube.com/watch?v=a4UF4C1hiB0&t=...
6,ben bu işte bu şekilde daha ne kadar devam ede...,7458,7462,bekir ben bu işte bu şekilde daha ne kadar dev...,QMUCHwM0CR4,https://www.youtube.com/watch?v=QMUCHwM0CR4&t=...
7,pekala pekala tamam tamam haydi kendi,2303,2306,pekala pekala tamam tamam haydi kendi işlerim var,Wp7JJK2jToU,https://www.youtube.com/watch?v=Wp7JJK2jToU&t=...


In [ ]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])
df_file_count.sort_values(by="word_count", ascending=False)
df_file_count

In [ ]:
df_check = df_file_count[df_file_count["word_count"] > 2]
df_check

In [ ]:
check_list = df_check["word"].to_list()
check_list

In [ ]:
len(check_list)

In [ ]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > 2]
check_list = df_check["word"].to_list()
remove_list = []
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file = df_file[df_file["search_string"] != f"{var_sent}"]
        df_file_count = word_count_result(df_file, df_file.columns[0:1])
        df_check = df_file_count[df_file_count["word_count"] > 2]
        check_list = df_check["word"].to_list()
        remove_list.append(var_sent)
    else:
        df_file = df_file

In [ ]:
df_file

In [ ]:
remove_list

In [ ]:
word_count_result(df_file, df_file.columns[0:1])

In [ ]:
test = ["aet","önce"]

In [ ]:
test in check_list

In [ ]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > 2]
check_list = df_check["word"].to_list()
i = 0
while i < len(df_file):
    var_sent in df_file.loc[i,"search_string"]
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file = df_file[df_file["search_string"] != f"{var_sent}"]
        df_file_count = word_count_result(df_file, df_file.columns[0:1])
        df_check = df_file_count[df_file_count["word_count"] > 1]
        check_list = df_check["word"].to_list()
    else:
        df_file = df_file
    i += 1

In [ ]:
df_file

In [ ]:
check_list

In [ ]:
len(check_list)

In [ ]:
while var_sent in df_file["search_string"]:
    print(var_sent)

In [ ]:
all_list = []
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    all_list.append(result_list)

In [ ]:
all_list

In [ ]:
test_list = []
for j in all_list:
    if all(j):
        test_list.append(j)
    else:
        pass

In [ ]:
test_list

In [ ]:
result_list

In [ ]:
all(result_list)

In [ ]:
a = []

In [ ]:
a.append(True)

In [ ]:
a.append(False)

In [ ]:
a